In [ ]:
import polars as pl

train = pl.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
test = pl.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')
display(train.head())
display(test.head())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(min_df=.05)
train_tfid = vectorizer.fit_transform([i for i in train['full_text']])
test_tfid = vectorizer.transform([i for i in test['full_text']])

train_y = np.array(train['score'])

In [ ]:
# !pip install flaml

In [ ]:
# from flaml import AutoML

# aml = AutoML()

# aml.fit(train_tfid, train_y, estimator_list = ['lgbm'], task = 'classification', metric = 'macro_f1', time_budget = 600)

In [ ]:
# aml.best_config

In [ ]:
import lightgbm as lgb

params = {'n_estimators': 195,
 'num_leaves': 6,
 'min_child_samples': 3,
 'learning_rate': 0.06518970520093895,
 'max_bin': 511,
 'colsample_bytree': 0.9699010403795221,
 'reg_alpha': 0.028218191448367104,
 'reg_lambda': 0.855160569748305}

model = lgb.LGBMClassifier(**params)
model.fit(train_tfid, train_y)

In [ ]:
submission = test.select('essay_id').with_columns(score = model.predict(test_tfid))
display(submission)
submission.write_csv('submission.csv')